In [1]:
import subprocess
import os
import subprocess
import re

def insert_logging_setup(repo_root):
    testbench_dir = os.path.join(repo_root, "val", "common_level0")

    for unit in os.listdir(testbench_dir):
        testbench_path = os.path.join(testbench_dir, unit, "testbench.py")
        if os.path.isfile(testbench_path):
            with open(testbench_path, "r") as file:
                lines = file.readlines()

            # Find the last import/from within the first 20 lines
            last_import_index = None
            for i, line in enumerate(lines[:20]):
                if line.startswith('import ') or line.startswith('from '):
                    last_import_index = i

            # Insert logging setup after the last import/from
            if last_import_index is not None:
                logging_setup = f"""
import pytest
from cocotb_test.simulator import run
import logging
log = logging.getLogger('cocotb_log_{unit}')
log.setLevel(logging.DEBUG)
# Create a file handler that logs even debug messages
fh = logging.FileHandler('cocotb_log_{unit}.log')
fh.setLevel(logging.DEBUG)
# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# Add the handler to the logger
log.addHandler(fh)

"""
                # Insert the logging setup
                lines.insert(last_import_index + 1, logging_setup)

                # Replace all print statements with log.info
                lines = [re.sub(r'^(\s*)print\((.*)\)', r'\1log.info(\2)', line, flags=re.M) for line in lines]

                # Write back the modified content
                with open(testbench_path, "w") as file:
                    file.writelines(lines)

# Example usage
repo_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')
insert_logging_setup(repo_root)
